In [2]:
import hrr_common as hrr_cm
import hrr_cobot_robot as hrr_rob
import rospy
import numpy as np
def get_hostname_and_IP():
    import socket
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
        s.connect(("8.8.8.8", 80))
        return socket.gethostname(), s.getsockname()[0]

_host, _ip = get_hostname_and_IP()
%env ROS_MASTER_URI=http://172.22.162.200:11311
print(f"\t\033[1m\033[4m!!!please check that the IPs below is identical!!!\033[0m"+
      f"\n\thost {_host} has IP {_ip}, which should be identical to",end="\nvs:\t")
%env ROS_IP=$_ip

env: ROS_MASTER_URI=http://172.22.162.200:11311
	!!!please check that the IPs below is identical!!!
	host hrrN3511rt2004 has IP 172.22.162.200, which should be identical to
vs:	env: ROS_IP=172.22.162.200


In [3]:
rospy.init_node("NotebookNode")

In [4]:
hrr_rob.load_default_parameters("/hrr_cobot")
cobot = hrr_rob.HrrCobotControl.from_ros()
print(cobot)

[ERROR]could not set tool-frame @ /NotebookNode due to missing ROS-parameter: '~tool_name'


[INFO]Updated force  gains: K_p:=[0.0001, 0.0001, 0.0001]
[INFO]Updated torque gains: K_p:=[0.0005, 0.0005, 0.0005]
Hrr-Cobot state:
FT-data:
current force:	[-0.  0.  0.] [N]
current torque:	[-0.  0.  0.] [Nm]
current wrench:	[0. 0. 0. 0. 0. 0.] [N,Nm]
=>in contact:	False
---
Robot-data:
q:		[-79.34705891  18.40579911 -84.79675073  -1.70305815 -47.55804712
   0.37251729][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[0.11634132 0.60901227 0.72172491][m]
quat:		0.68009 <[-0.28806108  0.36744603  0.56522871]>
tool-pos:	[0.11634132 0.60901227 0.72172491][m]
tool-rpy:	[ 84.69761658  10.02925873 -55.05702591][°]
robot-status:	ready
---


In [9]:
cobot.T_B_E_robot.A

array([[ 0.09099959, -0.98050515,  0.17415145,  0.11634148],
       [ 0.5571179 ,  0.19507482,  0.80719605,  0.60901227],
       [-0.82543244,  0.02356838,  0.5640087 ,  0.72172498],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [8]:
cobot.q

array([-1.38486743,  0.32124189, -1.4799825 , -0.02972397, -0.83004481,
        0.00650135])

### About the tool rack and this procedure
For all tools except the shaft grinder, the pickup-position stays the same if tools are swapped. Orientation matters however (last link).
For now assume that angle of joint1 < 0 refers to tool position 1,2,3 - where the angle of joint1 decreases from position 1 to 3. For joint1 > 0 we have positions 4,5,6 respectively.

The tool changer is open if digital out pin 5 is True, is 

In [23]:
!rosservice call /hrr_cobot/get_comau_parameters "{}"

move_fly: 
  threshold: 0.2
  lin_velocity: 0.20000000298023224
  fly_dist: 5.0
status: 
  status: "R"
sensor_track_config: 
  sensor_type: 7
  sensor_cnvrsn: 1
  sensor_gain: 100
  sensor_time: 0
  sensor_ofst_lim_trans: 500
  sensor_ofst_lim_rot: 800
digital_in_states: []
digital_out_states: 
  - 
    pin: 1
    state: True
  - 
    pin: 3
    state: False
  - 
    pin: 5
    state: False
  - 
    pin: 6
    state: False
  - 
    pin: 8
    state: True
  - 
    pin: 9
    state: False
  - 
    pin: 10
    state: False
  - 
    pin: 11
    state: True
  - 
    pin: 12
    state: False
  - 
    pin: 13
    state: False
  - 
    pin: 14
    state: False
  - 
    pin: 15
    state: False
  - 
    pin: 16
    state: False
is_async: False


In [21]:
from comau_msgs.srv import *
setPin = rospy.ServiceProxy("/hrr_cobot/set_digital_io",SetIO)

In [24]:
# Open tool changer
setPin(pin=5, state=True)
#!rosservice call /hrr_cobot/set_digital_io "pin: 5 state: true"

success: True

In [ ]:
# Close tool changer
setPin(pin=5, state=False)
#!rosservice call /hrr_cobot/set_digital_io "pin: 5 state: false"

#Vacuum gripper at position 1 - final position when closing the tool changer
cobot.update_tf()
np.save("rack_positions/rackpos_1_q", cobot.q, allow_pickle=True)
np.save("rack_positions/rackpos_1_A", cobot.T_B_E_robot.A, allow_pickle=True)

In [ ]:
cobot.update_tf()
np.save("rack_positions/rackpos_2_q", cobot.q, allow_pickle=True)
np.save("rack_positions/rackpos_2_A", cobot.T_B_E_robot.A, allow_pickle=True)

In [ ]:
cobot.update_tf()
np.save("rack_positions/rackpos_3_q", cobot.q, allow_pickle=True)
np.save("rack_positions/rackpos_3_A", cobot.T_B_E_robot.A, allow_pickle=True)

In [ ]:
cobot.update_tf()
np.save("rack_positions/rackpos_4_q", cobot.q, allow_pickle=True)
np.save("rack_positions/rackpos_4_A", cobot.T_B_E_robot.A, allow_pickle=True)

In [ ]:
cobot.update_tf()
np.save("rack_positions/rackpos_5_q", cobot.q, allow_pickle=True)
np.save("rack_positions/rackpos_5_A", cobot.T_B_E_robot.A, allow_pickle=True)

In [ ]:
cobot.update_tf()
np.save("rack_positions/rackpos_6_q", cobot.q, allow_pickle=True)
np.save("rack_positions/rackpos_6_A", cobot.T_B_E_robot.A, allow_pickle=True)